### Import Library

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### đọc dữ liệu

In [51]:
path = r'C:\Users\nguye\OneDrive\Tài liệu\GitHub\Edge-IIOT\ML-EdgeIIoT-dataset.csv'

In [52]:
data = pd.read_csv(path)

C:\Users\nguye\AppData\Local\Temp\ipykernel_14416\822284616.py:1: DtypeWarning: Columns (3,6,11,13,14,15,16,17,31,32,34,39,45,51,54,55) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path)


In [53]:
df = data.copy()

### EDA

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157800 entries, 0 to 157799
Data columns (total 63 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   frame.time                 157800 non-null  object 
 1   ip.src_host                157800 non-null  object 
 2   ip.dst_host                157800 non-null  object 
 3   arp.dst.proto_ipv4         157800 non-null  object 
 4   arp.opcode                 157800 non-null  float64
 5   arp.hw.size                157800 non-null  float64
 6   arp.src.proto_ipv4         157800 non-null  object 
 7   icmp.checksum              157800 non-null  float64
 8   icmp.seq_le                157800 non-null  float64
 9   icmp.transmit_timestamp    157800 non-null  float64
 10  icmp.unused                157800 non-null  float64
 11  http.file_data             157800 non-null  object 
 12  http.content_length        157800 non-null  float64
 13  http.request.uri.query     15

In [55]:
df.columns

Index(['frame.time', 'ip.src_host', 'ip.dst_host', 'arp.dst.proto_ipv4',
       'arp.opcode', 'arp.hw.size', 'arp.src.proto_ipv4', 'icmp.checksum',
       'icmp.seq_le', 'icmp.transmit_timestamp', 'icmp.unused',
       'http.file_data', 'http.content_length', 'http.request.uri.query',
       'http.request.method', 'http.referer', 'http.request.full_uri',
       'http.request.version', 'http.response', 'http.tls_port', 'tcp.ack',
       'tcp.ack_raw', 'tcp.checksum', 'tcp.connection.fin',
       'tcp.connection.rst', 'tcp.connection.syn', 'tcp.connection.synack',
       'tcp.dstport', 'tcp.flags', 'tcp.flags.ack', 'tcp.len', 'tcp.options',
       'tcp.payload', 'tcp.seq', 'tcp.srcport', 'udp.port', 'udp.stream',
       'udp.time_delta', 'dns.qry.name', 'dns.qry.name.len', 'dns.qry.qu',
       'dns.qry.type', 'dns.retransmission', 'dns.retransmit_request',
       'dns.retransmit_request_in', 'mqtt.conack.flags',
       'mqtt.conflag.cleansess', 'mqtt.conflags', 'mqtt.hdrflags', 'mqtt.len

In [56]:
df.duplicated().sum()

814

### Data Cleaning


In [57]:
drop_columns = ['ip.src_host', 'ip.dst_host','http.content_length', 'dns.qry.name.len', 'mqtt.len',
'mqtt.msg', 'mqtt.proto_len', 'mqtt.topic_len']
df.drop(drop_columns, axis = 1, inplace = True)

In [58]:
df.drop_duplicates(subset = None, keep = 'first', inplace = True) # loại bỏ các từ trùng lặp

In [59]:
df.duplicated().sum()

0

In [60]:
df.isnull().sum()

frame.time                   0
arp.dst.proto_ipv4           0
arp.opcode                   0
arp.hw.size                  0
arp.src.proto_ipv4           0
icmp.checksum                0
icmp.seq_le                  0
icmp.transmit_timestamp      0
icmp.unused                  0
http.file_data               0
http.request.uri.query       0
http.request.method          0
http.referer                 0
http.request.full_uri        0
http.request.version         0
http.response                0
http.tls_port                0
tcp.ack                      0
tcp.ack_raw                  0
tcp.checksum                 0
tcp.connection.fin           0
tcp.connection.rst           0
tcp.connection.syn           0
tcp.connection.synack        0
tcp.dstport                  0
tcp.flags                    0
tcp.flags.ack                0
tcp.len                      0
tcp.options                  0
tcp.payload                  0
tcp.seq                      0
tcp.srcport                  0
udp.port

In [61]:
df['Attack_label'].value_counts()

Attack_label
1    132680
0     24301
Name: count, dtype: int64

In [62]:
df['Attack_type'].unique()

array(['MITM', 'Fingerprinting', 'Ransomware', 'Uploading',
       'SQL_injection', 'DDoS_HTTP', 'DDoS_TCP', 'Password',
       'Port_Scanning', 'Vulnerability_scanner', 'Backdoor', 'XSS',
       'Normal', 'DDoS_UDP', 'DDoS_ICMP'], dtype=object)

In [63]:
mapping = {
    'DDoS_UDP': 'DoS/DDoS Attack',
    'DDoS_ICMP' : 'DoS/DDoS Attack',
    'DDoS_HTTP': 'DoS/DDoS Attack',
    'DDoS_TCP': 'DoS/DDoS Attack',
    'Port_Scanning': 'Information Gathering',
    'Fingerprinting': 'Information Gathering',
    'Vulnerability_scanner': 'Information Gathering',
    'MITM': 'Main in the middle Attack',
    'XSS': 'Injection Attack',
    'SQL_injection': 'Injection Attack',
    'Uploading': 'Injection Attack',
    'Backdoor': 'Malware Attack',
    'Password': 'Malware Attack',
    'Ransomware': 'Malware Attack',
    'Normal' : 'Normal'
}

df['Attack_type'] = df['Attack_type'].map(mapping)

### Feature Selection

In [64]:
df2 = df.copy()

In [65]:
df2.drop('frame.time',axis = 1,inplace = True)

In [66]:
df2.select_dtypes(include=['object']).info()

<class 'pandas.core.frame.DataFrame'>
Index: 156981 entries, 0 to 157799
Data columns (total 15 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   arp.dst.proto_ipv4      156981 non-null  object
 1   arp.src.proto_ipv4      156981 non-null  object
 2   http.file_data          156981 non-null  object
 3   http.request.uri.query  156981 non-null  object
 4   http.request.method     156981 non-null  object
 5   http.referer            156981 non-null  object
 6   http.request.full_uri   156981 non-null  object
 7   http.request.version    156981 non-null  object
 8   tcp.options             156981 non-null  object
 9   tcp.payload             156981 non-null  object
 10  tcp.srcport             156981 non-null  object
 11  mqtt.conack.flags       156981 non-null  object
 12  mqtt.protoname          156981 non-null  object
 13  mqtt.topic              156981 non-null  object
 14  Attack_type             156981 non-null  

In [67]:
df2['http.file_data'].value_counts()

http.file_data
0.0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

*cột http.file_data chứa nhiều dữ liệu dạng text*

In [68]:
df2.drop('http.file_data', axis = 1, inplace = True)

In [69]:
df2['tcp.srcport'].value_counts()

tcp.srcport
80.0      33569
0.0       17641
1883.0    10075
4321.0     4864
4321.0     4796
          ...  
7815.0        1
7814.0        1
7808.0        1
7804.0        1
3922.0        1
Name: count, Length: 33551, dtype: int64

In [70]:
df2['tcp.srcport'].unique()

array(['0', 'DESKTOP-UHF0SF2', 'DESKTOP-UHF0SF2.local', ..., 63006.0,
       3416.0, 3922.0], dtype=object)

**Có khoảng 367 giá trị '_googlecast._tcp.local', 'DESKTOP-UHF0SF2', 'DESKTOP-UHF0SF2.local' => có thể drop**

In [71]:
df2['http.request.uri.query'].value_counts()

http.request.uri.query
0.0                                                                                                                                      91445
0.0                                                                                                                                      45149
0                                                                                                                                        17636
path_prefix=http://cirt.net/rfiinc.txt?                                                                                                     69
Command=GetFolders&Type=File&CurrentFolder=%2F                                                                                              49
                                                                                                                                         ...  
id=9%27%29%20ORDER%20BY%201--%20myYC&Submit=Submitid=59                                                                

In [72]:
df2.drop('http.request.uri.query', axis = 1, inplace = True)

**http.request.uri.query cũng chứa dữ liệu dạng text**

In [73]:
df2['tcp.options'].value_counts()

tcp.options
0                           27399
0.0                         26728
0.0                         23181
020405b40101040201030307     1325
020405b40103030801010402     1171
                            ...  
0101080a9eb1b1739b241e8f        1
0101080a9b241e8a9eb1b16a        1
0101080a9eb1b17a9b241e8f        1
0101080a9b241ead9eb1b174        1
0101080ae76b014c7adff520        1
Name: count, Length: 73140, dtype: int64

In [74]:
df2.drop('tcp.options', axis = 1, inplace = True)

**cột tcp.options : các tùy chọn trong gói tin tcp có chứa dữ liệu dạng text**

In [75]:
df2['tcp.payload'].value_counts()

tcp.payload
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [76]:
df2.drop('tcp.payload', axis = 1, inplace = True)

**cột tcp.payload có nhiều dữ liệu dạng text chưa xưr lý được**

In [77]:
df2['http.request.full_uri'].value_counts()

http.request.full_uri
0                                                                                                                                                                     54084
0.0                                                                                                                                                                   52611
0.0                                                                                                                                                                   43112
http://192.168.0.128/dvwa/vulnerabilities/                                                                                                                              866
http://192.168.0.128/DVWA/login.php                                                                                                                                     678
                                                                                                                      

In [78]:
df2.drop('http.request.full_uri',axis = 1, inplace = True)

**cột http.request.full_uri có nhiều dữ liệu dangj text chưa xử lý được**

### Encode

In [79]:
df3 = df2.copy()

In [80]:
# Danh sách các giá trị cần loại bỏ
invalid_values = ['_googlecast._tcp.local', 'DESKTOP-UHF0SF2', 'DESKTOP-UHF0SF2.local']

# Lọc các giá trị không hợp lệ
df3['tcp.srcport'] = df3['tcp.srcport'].replace(invalid_values, np.nan)
df3.isnull().sum()

arp.dst.proto_ipv4             0
arp.opcode                     0
arp.hw.size                    0
arp.src.proto_ipv4             0
icmp.checksum                  0
icmp.seq_le                    0
icmp.transmit_timestamp        0
icmp.unused                    0
http.request.method            0
http.referer                   0
http.request.version           0
http.response                  0
http.tls_port                  0
tcp.ack                        0
tcp.ack_raw                    0
tcp.checksum                   0
tcp.connection.fin             0
tcp.connection.rst             0
tcp.connection.syn             0
tcp.connection.synack          0
tcp.dstport                    0
tcp.flags                      0
tcp.flags.ack                  0
tcp.len                        0
tcp.seq                        0
tcp.srcport                  367
udp.port                       0
udp.stream                     0
udp.time_delta                 0
dns.qry.name                   0
dns.qry.qu

**chuyển '_googlecast._tcp.local', 'DESKTOP-UHF0SF2', 'DESKTOP-UHF0SF2.local' thành nan và drop nan**

In [81]:
df3['tcp.srcport'].dropna()

0           0
8           0
29          0
84          0
153         0
         ... 
157795    0.0
157796    0.0
157797    0.0
157798    0.0
157799    0.0
Name: tcp.srcport, Length: 156614, dtype: object

In [82]:
df3.dropna(subset=['tcp.srcport'], inplace=True)

In [83]:
df3.isna().sum()

arp.dst.proto_ipv4           0
arp.opcode                   0
arp.hw.size                  0
arp.src.proto_ipv4           0
icmp.checksum                0
icmp.seq_le                  0
icmp.transmit_timestamp      0
icmp.unused                  0
http.request.method          0
http.referer                 0
http.request.version         0
http.response                0
http.tls_port                0
tcp.ack                      0
tcp.ack_raw                  0
tcp.checksum                 0
tcp.connection.fin           0
tcp.connection.rst           0
tcp.connection.syn           0
tcp.connection.synack        0
tcp.dstport                  0
tcp.flags                    0
tcp.flags.ack                0
tcp.len                      0
tcp.seq                      0
tcp.srcport                  0
udp.port                     0
udp.stream                   0
udp.time_delta               0
dns.qry.name                 0
dns.qry.qu                   0
dns.qry.type                 0
dns.retr

In [84]:
df3['tcp.srcport'] = df3['tcp.srcport'].astype('float64')

**gán dữ liệu cột tcp.srcport thành float64**

In [85]:
print(df3['tcp.srcport'].dtype)  # Kiểm tra kiểu dữ liệu

float64


In [86]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
LE = LabelEncoder()
df3['Attack_type'] = LE.fit_transform(df3['Attack_type'])

In [87]:
df3.select_dtypes(include=['object']).info()

<class 'pandas.core.frame.DataFrame'>
Index: 156614 entries, 0 to 157799
Data columns (total 8 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   arp.dst.proto_ipv4    156614 non-null  object
 1   arp.src.proto_ipv4    156614 non-null  object
 2   http.request.method   156614 non-null  object
 3   http.referer          156614 non-null  object
 4   http.request.version  156614 non-null  object
 5   mqtt.conack.flags     156614 non-null  object
 6   mqtt.protoname        156614 non-null  object
 7   mqtt.topic            156614 non-null  object
dtypes: object(8)
memory usage: 10.8+ MB


In [88]:
Columns = ['arp.dst.proto_ipv4', 'arp.src.proto_ipv4','http.request.method','http.referer', 'http.request.version', 'mqtt.conack.flags', 
           'mqtt.protoname', 'mqtt.topic']
mapping = list()
for column in Columns:
    df3[column] = df3[column].astype('str')
    df3[column] = LE.fit_transform(df3[column])
    mappings_dict = {index: label for index, label in enumerate(LE.classes_)}
    mapping.append(mappings_dict)

**Mã hóa 'arp.dst.proto_ipv4', 'arp.src.proto_ipv4','http.request.method','http.referer', 'http.request.version', 'mqtt.conack.flags', 'mqtt.protoname', 'mqtt.topic' theo label encoder**

In [89]:
mapping

[{0: '0',
  1: '0.0',
  2: '192.168.0.1',
  3: '192.168.0.101',
  4: '192.168.0.128',
  5: '192.168.0.129',
  6: '192.168.0.147',
  7: '192.168.0.170'},
 {0: '0',
  1: '0.0',
  2: '0.0.0.0',
  3: '192.168.0.1',
  4: '192.168.0.101',
  5: '192.168.0.128',
  6: '192.168.0.129',
  7: '192.168.0.170'},
 {0: '0', 1: '0.0', 2: 'GET', 3: 'OPTIONS', 4: 'POST', 5: 'TRACE'},
 {0: '() { _; } >_[$($())] { echo 93e4r0-CVE-2014-6278: true; echo;echo; }',
  1: '0',
  2: '0.0',
  3: '127.0.0.1'},
 {0: '-a HTTP/1.1',
  1: '/etc/passwd|?data=Download HTTP/1.1',
  2: '0',
  3: '0.0',
  4: 'By Dr HTTP/1.1',
  5: 'HTTP/1.0',
  6: 'HTTP/1.1',
  7: 'Src=javascript:alert(\'Vulnerable\')><Img Src=\\" HTTP/1.1'},
 {0: '0', 1: '0.0', 2: '0x00000000'},
 {0: '0', 1: '0.0', 2: 'MQTT'},
 {0: '0', 1: '0.0', 2: 'Temperature_and_Humidity'}]

In [90]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report, accuracy_score

### Chia dữ liệu


In [91]:
X = df3.iloc[:, 0:47]
y =  df3['Attack_type']

In [92]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [93]:
print(y_train.shape)

(125291,)


In [94]:
print(X_train.shape)

(125291, 47)


### Đánh giá mô hình


**scale dữ liệu**

In [95]:
from sklearn.preprocessing import StandardScaler
Ro_scaler = StandardScaler()
X_train = Ro_scaler.fit_transform(X_train) 
X_test= Ro_scaler.transform(X_test)

In [96]:
X_train.shape, y_train.shape

((125291, 47), (125291,))

In [99]:
data_train = np.concatenate((X_train, np.array(y_train).reshape(y_train.shape[0], 1)), axis = 1)
data_test = np.concatenate((X_test, np.array(y_test).reshape(y_test.shape[0], 1)), axis = 1)

In [100]:
np.save('train_data.npy',data_train)
np.save('test_data.npy',data_test)

**Mô hình Decision Tree**

In [106]:
DT = DecisionTreeClassifier()
DT.fit(X_train, y_train)

DecisionTreeClassifier()

In [107]:
def y_pred(Model_app, X_test) :
    y_predict = Model_app.predict(X_test)
    return y_predict

In [108]:
def Static_score_model_class(y, y_pred,aver = 'macro'):
    accuracy = accuracy_score(y, y_pred)
    precision = precision_score(y,y_pred, average = aver)
    recall = recall_score(y,y_pred, average = aver)
    f1 = f1_score(y, y_pred, average = aver)
    print("Accuracy score: {} \n".format(accuracy))
    print("Precision score: {} \n".format(precision))
    print("Recall score: {} \n".format(recall))
    print("F1 score: {} \n".format(f1))
    print(classification_report(y,y_pred))

In [109]:
Static_score_model_class(y_test, y_pred(DT, X_test), aver = 'macro')

Accuracy score: 0.9588162053443157 

Precision score: 0.9690122442167789 

Recall score: 0.9665090381579554 

F1 score: 0.9676897004187069 

              precision    recall  f1-score   support

           0       0.96      0.97      0.96      9802
           1       1.00      0.96      0.98      4250
           2       0.94      0.93      0.93      6114
           3       1.00      1.00      1.00         5
           4       0.92      0.94      0.93      6133
           5       1.00      1.00      1.00      5019

    accuracy                           0.96     31323
   macro avg       0.97      0.97      0.97     31323
weighted avg       0.96      0.96      0.96     31323



**Mô hình KNN**

In [110]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier()
KNN.fit(X_train,y_train)

KNeighborsClassifier()

In [111]:
Static_score_model_class(y_test,y_pred(KNN, X_test))

Exception in thread Thread-37 (_readerthread):
Traceback (most recent call last):
  File "c:\Users\nguye\anaconda3\envs\iot\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "c:\Users\nguye\anaconda3\envs\iot\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\nguye\anaconda3\envs\iot\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\nguye\anaconda3\envs\iot\Lib\subprocess.py", line 1599, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
  File "<frozen codecs>", line 322, in decode
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xdd in position 35: invalid continuation byte
c:\Users\nguye\anaconda3\envs\iot\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
'NoneType' object has no attribute 'splitlines'
Returning the number of 

Accuracy score: 0.9057880790473454 

Precision score: 0.9278103079183078 

Recall score: 0.925398168550526 

F1 score: 0.9258000861465333 

              precision    recall  f1-score   support

           0       0.91      0.92      0.92      9802
           1       0.96      0.96      0.96      4250
           2       0.80      0.88      0.84      6114
           3       1.00      1.00      1.00         5
           4       0.90      0.80      0.84      6133
           5       1.00      1.00      1.00      5019

    accuracy                           0.91     31323
   macro avg       0.93      0.93      0.93     31323
weighted avg       0.91      0.91      0.91     31323



**Mô hình XGBoost**

In [112]:
from xgboost import XGBClassifier
XG = XGBClassifier()
XG.fit(X_train,y_train)

ModuleNotFoundError: No module named 'xgboost'

In [51]:
Static_score_model_class(y_test, y_pred(XG, X_test))

Accuracy score: 0.9654186034632477 

Precision score: 0.9649139207039064 

Recall score: 0.9547474651485842 

F1 score: 0.9588225459328854 

              precision    recall  f1-score   support

           0       1.00      0.97      0.98      2500
           1       0.82      0.90      0.86      2621
           2       1.00      1.00      1.00      3480
           3       1.00      1.00      1.00      2479
           4       1.00      1.00      1.00      3615
           5       0.97      0.83      0.89       260
           6       1.00      1.00      1.00         6
           7       1.00      1.00      1.00      6245
           8       0.96      0.83      0.89      2575
           9       0.99      0.94      0.96      2550
          10       0.91      1.00      0.95      2649
          11       0.89      0.89      0.89      2630
          12       0.93      0.97      0.95      2499
          13       1.00      1.00      1.00      2490
          14       1.00      1.00      1.00     

**Mô hình RandomForest**

In [113]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier()
RF.fit(X_train, y_train)

RandomForestClassifier()

In [114]:
Static_score_model_class(y_test,y_pred(RF,X_test))

Accuracy score: 0.9643073779650736 

Precision score: 0.973483209692147 

Recall score: 0.9707187892532835 

F1 score: 0.9719932101087604 

              precision    recall  f1-score   support

           0       0.96      0.97      0.97      9802
           1       1.00      0.96      0.98      4250
           2       0.93      0.96      0.94      6114
           3       1.00      1.00      1.00         5
           4       0.95      0.93      0.94      6133
           5       1.00      1.00      1.00      5019

    accuracy                           0.96     31323
   macro avg       0.97      0.97      0.97     31323
weighted avg       0.96      0.96      0.96     31323



In [54]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [55]:
# Tạo mô hình Sequential
model = Sequential()

# Thêm lớp đầu vào với số lượng đặc trưng bằng số cột đầu vào (47)
model.add(Dense(units=128, activation='relu', input_shape=(47,)))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=32, activation='relu'))
# Lớp đầu ra với 15 lớp và hàm kích hoạt softmax
model.add(Dense(units=15, activation='softmax'))

c:\Users\nguye\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [56]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [59]:
# Huấn luyện mô hình
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.2)

Epoch 1/30
2937/2937 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8015 - loss: 0.9124 - val_accuracy: 0.8002 - val_loss: 0.5001
Epoch 2/30
2937/2937 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.7994 - loss: 0.7920 - val_accuracy: 0.7924 - val_loss: 0.5901
Epoch 3/30
2937/2937 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.8039 - loss: 0.7227 - val_accuracy: 0.8017 - val_loss: 0.4886
Epoch 4/30
2937/2937 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8003 - loss: 1.3585 - val_accuracy: 0.8003 - val_loss: 0.4839
Epoch 5/30
2937/2937 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.8050 - loss: 2.2569 - val_accuracy: 0.8020 - val_loss: 0.6557
Epoch 6/30
2937/2937 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.8013 - loss: 3.0998 - val_accuracy: 0.8016 - val_loss: 0.6674
Epoch 7/30
2937/2937 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.8075 - loss: 1.0362 - val_accuracy: 0.8020 - val_loss: 0.5145
Epoch 8/30
2937/2937 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.8055 - loss: 0.6279 - 

In [60]:
# Đánh giá mô hình
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

1224/1224 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7503 - loss: 0.6498
Test Loss: 0.669485867023468
Test Accuracy: 0.7486591339111328


In [117]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
X_train_tensor = torch.tensor(X_train, dtype = torch.float32)
y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.long)

In [121]:
X_train.shape

(125291, 47)

In [118]:
# Create PyTorch dataset and dataloader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [122]:
# define the model
class DNN(nn.Module) :
    def __init__(self) :
        super(DNN, self).__init__()
        self.fc1 = nn.Linear(47, 128)
        self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(64, 32)
        self.dropout3 = nn.Dropout(0.2)
        self.fc4 = nn.Linear(32, 6)
        self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = nn.ReLU()(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        x = nn.ReLU()(x)
        x = self.dropout3(x)
        x = self.fc4(x)
        x = self.softmax(x)
        return x
    
# Initialize the model, optimizer, and loss function
model = DNN()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
# Training loop

num_epochs = 100

for epoch in range(num_epochs):
    # Training
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {running_loss/len(train_loader):.4f}')

    # Validation
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print(f'Epoch [{epoch+1}/{num_epochs}], Test Accuracy: {100 * correct / total:.2f}%')

Epoch [1/100], Train Loss: 1.2614
Epoch [1/100], Test Accuracy: 82.09%
Epoch [2/100], Train Loss: 1.2171
Epoch [2/100], Test Accuracy: 83.11%
Epoch [3/100], Train Loss: 1.2125
Epoch [3/100], Test Accuracy: 83.18%
Epoch [4/100], Train Loss: 1.2116
Epoch [4/100], Test Accuracy: 83.05%
Epoch [5/100], Train Loss: 1.2114
Epoch [5/100], Test Accuracy: 83.06%
Epoch [6/100], Train Loss: 1.2110
Epoch [6/100], Test Accuracy: 83.05%
Epoch [7/100], Train Loss: 1.2112
Epoch [7/100], Test Accuracy: 83.10%
Epoch [8/100], Train Loss: 1.2106
Epoch [8/100], Test Accuracy: 82.85%
Epoch [9/100], Train Loss: 1.2106
Epoch [9/100], Test Accuracy: 83.26%
Epoch [10/100], Train Loss: 1.2124
Epoch [10/100], Test Accuracy: 83.15%
Epoch [11/100], Train Loss: 1.2104
Epoch [11/100], Test Accuracy: 83.31%
Epoch [12/100], Train Loss: 1.2095
Epoch [12/100], Test Accuracy: 83.21%
Epoch [13/100], Train Loss: 1.2095
Epoch [13/100], Test Accuracy: 83.22%
Epoch [14/100], Train Loss: 1.2096
Epoch [14/100], Test Accuracy: 83.

In [123]:
import time
torch.save(model.state_dict(),'model_weights.pth')

In [124]:
start_time = time.time()
model.load_state_dict(torch.load('model_weights.pth'))
model.eval()
output = model(X_test_tensor[:1])
end_time = time.time()
print(end_time - start_time)

0.16441917419433594


C:\Users\nguye\AppData\Local\Temp\ipykernel_14416\1052192933.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_weights.pth'))
